In [1]:
%load_ext lab_black
%cd ..

/home/shim/cev/dl/motion-keypoint


In [2]:
import math
import os
import random
import shutil
import json
import logging
import sys
from collections import defaultdict
from datetime import datetime
from io import TextIOWrapper
from pathlib import Path
from typing import Callable, List, Sequence, Tuple

import albumentations as A
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.tensorboard import SummaryWriter
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from tqdm import tqdm
from PIL import Image

import utils
import networks
from error_list import error_list

In [3]:
train_imgs = sorted(list(Path("data/box_effdet/train_imgs/").glob("*.jpg")))
test_imgs = sorted(list(Path("data/box_effdet/test_imgs/").glob("*.jpg")))

In [4]:
imgs = train_imgs + test_imgs

In [5]:
mean = [0, 0, 0]
for f in tqdm(imgs, ncols=100, file=sys.stdout):
    img = imageio.imread(f).astype(np.float64) / 255.0
    mean[0] += img[..., 0].mean()
    mean[1] += img[..., 1].mean()
    mean[2] += img[..., 2].mean()

100%|██████████████████████████████████████████████████████████| 5795/5795 [00:21<00:00, 266.70it/s]


In [6]:
mean[0] /= len(imgs)
mean[1] /= len(imgs)
mean[2] /= len(imgs)

In [7]:
mean

[0.4112477475044492, 0.41951101162188087, 0.41573220158294094]

In [10]:
std = [0, 0, 0]
for f in tqdm(imgs, ncols=100, file=sys.stdout):
    img = imageio.imread(f).astype(np.float64) / 255.0
    std[0] += np.square(img[..., 0] - mean[0]).mean()
    std[1] += np.square(img[..., 1] - mean[1]).mean()
    std[2] += np.square(img[..., 2] - mean[2]).mean()

100%|██████████████████████████████████████████████████████████| 5795/5795 [00:23<00:00, 244.73it/s]


In [11]:
std[0] /= len(imgs)
std[1] /= len(imgs)
std[2] /= len(imgs)

In [12]:
std

[0.09411564645816416, 0.09179505676253656, 0.08536205586972648]

In [13]:
std[0] = np.sqrt(std[0])
std[1] = np.sqrt(std[1])
std[2] = np.sqrt(std[2])

In [14]:
std

[0.30678273494146335, 0.3029769904836613, 0.29216785564077113]